In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from pprint import pprint
import snoop

Во-первых, для меня это возможность углубить свой преподавательский опыт и поделиться уже имеющимся, плюс мне очень интересно ознакомиться с программой Яндекс.Практикума. В дальнейшем хотелось бы ревьюить также работы с потока не только аналитиков, но и дата-сайентистов.

1. Определите пересекающуюся аудиторию в рамках групп treatment и control. Какой размер пересекающейся аудитории?

In [95]:
df1, df2 = pd.read_csv('ab_data.csv'), pd.read_csv('revenue_data.csv')

In [3]:
df1.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [96]:
t = set(df1[df1['group']=='treatment']['user_id'])
c = set(df1[df1['group']=='control']['user_id'])
intersection = t & c
len(intersection)

1895

In [123]:
t = df1.set_index('user_id').join(df2.set_index('user_id'), on='user_id')
t.reset_index()

,user_id,timestamp,group,landing_page,converted,revenue
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,10.284548
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,7.649273
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,11.909644
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,23.693355
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,19.996224
...,...,...,...,...,...,...
290683,751197,2017-01-03 22:28:38.630509,control,old_page,0,17.600421
290684,945152,2017-01-12 00:51:57.078372,control,old_page,0,10.936073
290685,734608,2017-01-22 11:45:03.439544,control,old_page,0,8.701321
290686,697314,2017-01-15 01:20:28.957438,control,old_page,0,19.623677


In [126]:
sum(t[t['group']=='treatment']['revenue']), sum(t[t['group']=='control']['revenue'])

(1743984.959263918, 1745055.1948785384)

In [132]:
import scipy

scipy.stats.mannwhitneyu(
    t[t["group"] == "treatment"]["revenue"], t[t["group"] == "control"]["revenue"], alternative='less')

MannwhitneyuResult(statistic=10533746341.0, pvalue=0.10233488468531643)

In [135]:
s1 = pd.Series(data=[1, 2], index=['a', 'b']) # можем вызвать s1['a']
print(s1)
print(s1['a'])

a    1
b    2
dtype: int64
1


In [137]:
s2 = pd.Series(data=[3, 4], index=['a', 'b'])
print(s2)
print(s2['a'])

a    3
b    4
dtype: int64
3


Во-первых, Series это одномерная структура данных представляющая из себя набор пар "ключ-значение", где ключи могут выступать в роли индексов. Если объединить несколько series с одинаковыми ключами (индексами) то получится DataFrame, двумерная структура данных – таблица, в которой бывшие индексы будут названиями столбцов. 
https://dsc.cloud/999174/pb-NNpRObM3R0.png

In [144]:
s3 = pd.DataFrame(data=[s1, s2])
s3

,a,b
0,1,2
1,3,4


In [97]:
df1 = df1.set_index('user_id').drop(intersection, axis=0)
# len(df1)
df1 = df1.reset_index()

Очистите данные от сессий пользователей, которые входят в пересекающуюся аудиторию. Все дальнейшие манипуляции производите с очищенными данными

Какое количество уникальных пользователей в группе control?

In [35]:
c = set(df1[df1['group']=='control'].index)
len(c)

144300

In [99]:
temp = df1.groupby('timestamp')['user_id'].count()

In [105]:
temp=df1.groupby(['user_id'])['timestamp'].count()

In [112]:
len(temp[temp==2])

1999

In [32]:
df1[df1['group']=='control'].index

Int64Index([851104, 804228, 864975, 936923, 719014, 644214, 847721, 650559,
            935734, 746742,
            ...
            834362, 923948, 857744, 755438, 908354, 718310, 751197, 945152,
            734608, 697314],
           dtype='int64', name='user_id', length=145307)

Какое количество пользователей совершили более чем 1 сессии за время наблюдений?

In [41]:
g = df1.reset_index()
g = g.groupby(['user_id']).count()['timestamp']
g.value_counts()

Конверсия рассчитывается как отношение количества сессий к количеству сессий, в которых произошла конверсия (колонка converted). Какая нижняя граница доверительного интревала для конверсии при условии, что alpha = 0.05? (Ответ дайте с точностью до 3 знаков после запятой, используйте математическое округление. Ответ - число в виде десятичной дроби, а не процентного числа)

In [64]:
mean_conv = len(df1[df1['converted']==1])/len(df1)

In [74]:
from statsmodels.stats.proportion import proportion_confint

In [75]:
proportion_confint(sum(df1['converted']), len(df1), method='normal')

(0.1184570898267346, 0.12081663320276782)

In [ ]:
df3['budget'].apply(lambda x: )

In [83]:
df3 = pd.read_csv('movie_metadata.csv')

Загрузите датасет movie_metadata.csv, который содержит данные о фильмах из базы IMDb (Internet Movie Database). Колонка duration - данные о длительности фильма.

Сколько пропущенных значений содержится в этой колонке?

In [97]:
df3.duration.fillna(value=np.mean(df3.duration), inplace=True)

Создайте колонку movie_duration_category, которая содержала бы 3 категории в зависимости от длительности фильма:

Категория "1. < 90", если фильм длится менее 90 минут;
Категория "2. 90-120", если фильм длится от 90 минут до 2 часов (включительно);
Категория "3. > 120", если фильм длится больше 2 часов.
Постройте сводную таблицу для фильмов с годом выпуска после 2000 (включительно), которая отражала бы количество фильмов:

В строках таблицы - год;
В столбцах таблицы - категория длительности фильма ("< 90", "90-120", "> 120");
Год фильма должен быть представлен в формате YYYY
Сколько фильмов с длительностью от 90 минут до 2 часов было выпущено в 2008 году?

In [48]:
df3['mov_dur_cat'] = pd.cut(df3.duration, bins=[-np.inf, 90, 120, np.inf])

In [78]:
t = df3.groupby(['title_year', 'mov_dur_cat']).count()

In [87]:
np.mean(df3['budget'].astype('float'))

35874316.003569305

In [85]:
df3['budget'] = df3.budget.apply(lambda x: x[:-1])

In [138]:
df3.budget = df3.budget.apply(lambda x: x[:-3])

In [3]:
# df3['plot_keywords'][0].split('|')
# @snoop
def foo(keywords):
    if keywords is not df3.plot_keywords[4]:
        keywords = keywords.split('|')
        if ['love', 'death' ] in keywords: return 'love_and_death'
        elif 'love' in keywords: return 'love'
        elif 'death' in keywords: return 'death'
        else: return 'other'
    else:
        return 'other'
df3['mov_cat'] = df3['plot_keywords'].apply(lambda x: foo(x))

In [9]:
df3.groupby('mov_cat').mean()['imdb_score']

mov_cat
death    6.508130
love     6.540404
other    6.436298
Name: imdb_score, dtype: float64

In [5]:
df3[df3['mov_cat']=='love']

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,mov_cat
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,English,UK,PG,250000000.0$,2009.0,11000.0,7.5,2.35,10000,love
26,Color,James Cameron,315.0,194.0,0.0,794.0,Kate Winslet,29000.0,658672302.0,Drama|Romance,...,English,USA,PG-13,200000000.0$,1997.0,14000.0,7.7,2.35,26000,love
104,Color,Jon Turteltaub,235.0,109.0,226.0,370.0,Omar Benson Miller,12000.0,63143812.0,Action|Adventure|Family|Fantasy,...,English,USA,PG,150000000.0$,2010.0,418.0,6.1,2.35,11000,love
233,Color,James L. Brooks,168.0,121.0,274.0,157.0,Domenick Lombardozzi,273.0,30212620.0,Comedy|Drama|Romance,...,English,USA,PG-13,120000000.0$,2010.0,216.0,5.3,1.85,0,love
309,Color,Joel Schumacher,144.0,121.0,541.0,680.0,Rene Auberjonois,920.0,184031112.0,Action|Adventure|Fantasy,...,English,USA,PG-13,100000000.0$,1995.0,710.0,5.4,1.85,0,love
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,Color,Dan Curtis,NaN,99.0,45.0,224.0,Campbell Scott,1000.0,NaN,Fantasy|Romance,...,English,USA,Unrated,0.0$,1998.0,393.0,7.4,1.33,515,love
4930,Color,Gary Winick,91.0,78.0,56.0,184.0,Aaron Stanford,376.0,2882062.0,Comedy|Drama|Romance,...,English,USA,PG-13,150000.0$,2000.0,346.0,6.3,1.85,132,love
4942,Color,Paul Schrader,130.0,93.0,261.0,697.0,Ruby Dee,783.0,NaN,Fantasy|Horror|Thriller,...,English,USA,R,18000000.0$,1982.0,782.0,6.1,1.85,0,love
4984,Color,Edward Burns,36.0,98.0,0.0,73.0,Michael McGlone,138.0,10246600.0,Comedy|Drama|Romance,...,English,USA,R,25000.0$,1995.0,111.0,6.6,1.85,265,love


Даны две случайные величины X и Y. Про них известно, что:
E X = 0.5, D X = 2
E Y = 7, D Y = 3.5
cov (X, Y) = -0.8
Найдите дисперсию случайной величины Z = 2 X - 3 Y (верное решение будет между 45 и 50).

$$Var_{X-Y}=Var^2_X+Var^2_Y-2Cov_{XY}$$
$$Var_X=E[X^2]-E[X]^2$$


$$
\begin{aligned}
&\operatorname{Var}(a X+b Y)=a^{2} \operatorname{Var}(X)+b^{2} \operatorname{Var}(Y)+2 a b \operatorname{Cov}(X, Y) \\
&\operatorname{Var}(a X-b Y)=a^{2} \operatorname{Var}(X)+b^{2} \operatorname{Var}(Y)-2 a b \operatorname{Cov}(X, Y)
\end{aligned}
$$

In [39]:
import sympy
from sympy import Symbol as sym
var_x, var_y = sympy.Symbol("var_x"), sympy.Symbol("var_y")
var_z = sympy.Symbol("var_z")
a, b = sympy.Symbol("a"), sympy.Symbol('b')
cov_xy = sympy.Symbol("cov_xy")

In [36]:
E_X2, E2_X = sym('E_X2'), sym('E2_X')

In [37]:
var_x = E_X2 - E2_X

In [42]:
var_z = a*a*var_x+b**2*var_y-2*a*b*cov_xy

In [41]:
var_z

var_z

In [43]:
var_z.subs([(a, 2), (b, 3), (var_x, 2), (var_y, 3.5), (cov_xy, -0.8)])


49.1000000000000